### 中醫醫案症狀詞抽取
- model: CRF
- 舌象,脈象
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子

In [1]:
import json
import nltk

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len,stype) in ne_index_list: 
        if i == ne_idx:
            return "B-"+stype
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-"+stype
        elif i == (ne_idx + ne_len - 1):
            return "E-"+stype
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "tongue" in keys:
        main = summary_json["tongue"]
        main_list = getTagListWithString(main)
        #print(main_list)
    
    if "pulse" in keys:
        second = summary_json["pulse"]
        second_list = getTagListWithString(second)
        #print(second_list)  
        
    for sent in sentences:
        ne_index_list = []
        
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne),"T"))

        for ne in second_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne),"P"))        
                
        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)
        

print(len(docs))

48901


In [5]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [6]:
print(len(data))

48901


In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 21604
Seconds required: 2.793

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 238145.050752
Feature norm: 1.000000
Error norm: 483803.567677
Active features: 21602
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.471

***** Iteration #2 *****
Loss: 186579.709839
Feature norm: 1.134584
Error norm: 268920.762737
Active features: 21352
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.746

***** Iteration #3 *****
Loss: 154016.696860
Feature norm: 1.302415
Error norm: 110848.589824
Active features: 21287
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #41 *****
Loss: 10471.812446
Feature norm: 31.155897
Error norm: 2741.118723
Active features: 7561
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.829

***** Iteration #42 *****
Loss: 10259.095729
Feature norm: 32.555557
Error norm: 2479.690501
Active features: 7531
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.921

***** Iteration #43 *****
Loss: 9997.630894
Feature norm: 34.545688
Error norm: 1652.900329
Active features: 7519
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.913

***** Iteration #44 *****
Loss: 9755.535855
Feature norm: 37.311322
Error norm: 2042.859256
Active features: 7465
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.833

***** Iteration #45 *****
Loss: 9605.747270
Feature norm: 39.791125
Error norm: 2864.750731
Active features: 7493
Line search trials: 1
Line search step: 1.000000
Seconds

***** Iteration #85 *****
Loss: 8108.648522
Feature norm: 61.890023
Error norm: 1584.618439
Active features: 6169
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.826

***** Iteration #86 *****
Loss: 8102.913503
Feature norm: 62.054392
Error norm: 1279.020476
Active features: 6156
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.763

***** Iteration #87 *****
Loss: 8097.432062
Feature norm: 62.414439
Error norm: 1185.123475
Active features: 6134
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.746

***** Iteration #88 *****
Loss: 8092.513798
Feature norm: 62.539557
Error norm: 1033.155621
Active features: 6117
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.755

***** Iteration #89 *****
Loss: 8088.345578
Feature norm: 62.858806
Error norm: 1223.892600
Active features: 6073
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #128 *****
Loss: 7990.222304
Feature norm: 68.903220
Error norm: 367.464217
Active features: 5678
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.487

***** Iteration #129 *****
Loss: 7989.309344
Feature norm: 69.051914
Error norm: 949.715779
Active features: 5679
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.488

***** Iteration #130 *****
Loss: 7987.110597
Feature norm: 69.124473
Error norm: 366.833746
Active features: 5678
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.485

***** Iteration #131 *****
Loss: 7986.391003
Feature norm: 69.265597
Error norm: 943.442282
Active features: 5664
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 1.498

***** Iteration #132 *****
Loss: 7984.332016
Feature norm: 69.308405
Error norm: 293.805573
Active features: 5668
Line search trials: 2
Line search step: 0.500000
Seconds r

***** Iteration #170 *****
Loss: 7930.988204
Feature norm: 72.636757
Error norm: 504.972127
Active features: 5525
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.743

***** Iteration #171 *****
Loss: 7930.905677
Feature norm: 72.714528
Error norm: 1024.565699
Active features: 5513
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.748

***** Iteration #172 *****
Loss: 7929.066868
Feature norm: 72.732075
Error norm: 406.058592
Active features: 5517
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.742

***** Iteration #173 *****
Loss: 7928.642231
Feature norm: 72.795673
Error norm: 868.533532
Active features: 5506
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.748

***** Iteration #174 *****
Loss: 7927.111569
Feature norm: 72.814570
Error norm: 441.377692
Active features: 5508
Line search trials: 1
Line search step: 1.000000
Seconds 

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

( (O)
2 (O)
) (O)
金 (O)
龙 (O)
胶 (O)
囊 (O)
2 (O)
瓶 (O)
， (O)
每 (O)
次 (O)
1 (O)
粒 (O)
， (O)
每 (O)
日 (O)
2 (O)
次 (O)
， (O)
饭 (O)
后 (O)
服 (O)


In [12]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-T": 1,"I-T": 2,"E-T": 3,"B-P":4,"I-P":5,"E-P":6}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-T","I-T","E-T","B-P","I-P","E-P"]))

             precision    recall  f1-score   support

          O       1.00      0.99      1.00    316255
        B-T       0.63      0.80      0.71       926
        I-T       0.61      0.79      0.69      1285
        E-T       0.62      0.78      0.69       926
        B-P       0.67      0.80      0.73       610
        I-P       0.67      0.78      0.72       726
        E-P       0.68      0.81      0.74       609

avg / total       0.99      0.99      0.99    321337

